In [33]:
from encodec import EncodecModel
from encodec.utils import convert_audio
import torchaudio
import torch
import glob
from IPython.lib.display import Audio

In [4]:
data_paths = glob.glob('/home/jbajor/Dev/datasets/LibriLight/small/**/*.flac', recursive=True)

In [79]:
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(6.0)

flac, sr = torchaudio.load(data_paths[8])
flac = flac.unsqueeze(0)
flac = convert_audio(flac, sr, model.sample_rate, model.channels)

In [80]:
with torch.no_grad():
    encoded_frames = model.encode(flac)
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)

In [81]:
codes[0,1,:].max()

save = 5

for i in range(encoded_frames[0][0].shape[1]-save):
    encoded_frames[0][0][0,i+save,:] = torch.randint(low=0, high=1023, size=(encoded_frames[0][0].shape[2],))

In [82]:
decoded_output = model.decode(encoded_frames).squeeze()

In [83]:
torchaudio.save('/home/jbajor/Dev/datasets/LibriLight/test.wav', decoded_output.unsqueeze(0), model.sample_rate)